# 10.無関係の下位問題を抽出する。

> 無関係の下位問題を積極的に見つけて抽出すること
> 関数やコードブロックを見て、このコードの目標は何かを自問する
> 各行に対し、目標に直接効果のあるコードか、あるいは無関係の下位問題を解決しているか自問する
> 無関係な下位問題を解決しているコードが相当量あれば、それらは別関数にする


「下位問題」とは、そのコードブロックの目標とは直接的に関係ない、補助処理のようなこと指す。
* 正規表現のテスト、配列⇒文字列への置換など

# 10.1 入門例：findClosestLocation()

例えば、与えられた地点から最も近い地点を特定する関数`findClosestLocation()`を考える。

```js
const findClosestLocation = (lat, long, array) => {
    let closest = array[0];
    let closestDist = Number.MAX_VALUE;
    for(let i = 0; i < array.length; i++) {
        //２地点をラジアンに変換する
        const latRad = radians(lat);
        const longRad = radians(long);
        const latRad2 = radians(array[i].lat);
        const longRad2 = radians(array[i].long);

        //余弦定理より、２地点間の距離を求める
        const dist = Math.acos(Math.sin(latRad) * Math.sin(latRad2) + Math.cos(latRad) * Math.cos(latRad2) * Math.cos(longRad - longRad2)) * 6371;

        if(dist < closestDist) {
            closest = array[i];
            closestDist = dist;
        }
    }
    return closest;
}
```
この関数の目標は「与えられた地点から最も近い地点を見つける」のに対し、for内部では「ラジアン変換」や「距離の算出」といった、最終目標とはある種無関係な処理をしている。このような場合、無関係な処理を別関数に切り出すことで、コードから関数の目標をより明確にすることができる。

```js
const sphericalDistance = (lat1, long1, lat2, long2) => {
    const latRad = radians(lat1);
    const longRad = radians(long1);
    const latRad2 = radians(lat2);
    const longRad2 = radians(long2);

    return Math.acos(Math.sin(latRad) * Math.sin(latRad2) + Math.cos(latRad) * Math.cos(latRad2) * Math.cos(longRad - longRad2)) * 6371;
}

const findClosestLocation = (lat, long, array) => {
    let closest = array[0];
    let closestDist = Number.MAX_VALUE;
    for(let i = 0; i < array.length; i++) {
        //GOOD:無関係な処理を別関数に切り出すことで、関数の目標を明確になった
        const dist = sphericalDistance(lat, long, array[i].lat, array[i].long);
        if(dist < closestDist) {
            closest = array[i];
            closestDist = dist;
        }
    }
    return closest;
}
```
また、`sphericalDistance()`は別関数に切り出したことで、再利用可能な、単体でテスト可能な関数になった。
これにより、コード全体の保守性が向上する。

# 10.2 ユーティリティーコード & 10.3 汎用コード

例えばファイルを読み込むような関数の場合、pythonなどでは予めそのような機能が言語レベルで提供されていて、そのような再利用可能なコードをユーティリティーコードと呼ぶ。

```python
with open('file.txt') as f:
    for line in f:
        print(line)
```
C++の場合は、そのようなユーティリティコードは存在しないので、自分で実装する必要がある。

```cpp
ifstream file("file.txt");

//ファイルサイズの計算→ファイルサイズ分のメモリを確保
file.seekg(0, ios::end);
int size = file.tellg();
char* file_buf = new char[size];
//バッファにファイルを読み込む
file.seekg(0, ios::beg);
file.read(file_buf, size);
file.close();
```

このように、単に「ファイルを読む」という行為を行うために、それ以外の無関係なコードが乱立する場合、その部分を関数化し、再利用するのが良い。

また、別関数化するということは、コードを独立させる事に等しい。コードが独立していれば、その他の小さいタスクごとに区切られているので、大きなメソッドや関数の保守・改良が楽に行える。


## 10.4 汎用コードをたくさん作る

先に示した別関数化のように、どのオブジェクトなどの処理でも使うような汎用的な関数をたくさん作るのは望ましい。
特に、このようなコードには、`utils`といったとどのオブジェクト作成の時でも利用可能な関数をまとめたファイルを作るのが良い。

こうすることで、関数をライブラリ化でき、プロジェクト・オブジェクトごとの依存性を断ち切る事ができる。


## 10.5 プロジェクトに特化した機能

あるオブジェクトを生成し、そのオブジェクトに紐解く様々なプロパティやメソッドを利用するまえに、型変換や正規表現テスト、文字列置換などのいわゆる「下位問題」は別関数に切り出し、生成したオブジェクトの参照、操作を一か所のブロックにまとめると、よりよいコードになる。

ただし、特定のワードを検出するための正規表現などは、同じオブジェクトの定義内に残しておくと、関連が分かりやすくなるので、可読性の観点から切り出す必要はない。


## 10.6 既存のインタフェースを簡潔にする

javascriptのWebAPIに見られるような、パラメータの設定や値代入の仕方が分かりずらいようなものを利用する場合、
ラッパー関数を自身で作成し、複雑な形式を吸収し、「引数に値をしてする」という簡単な方法に落とし込むことができる。

```js
//例：cookieの扱い
let max_results;
const cookies = document.cookie.split(";") 
//name1=value1;name2=value2....のようにすべてが連結された文字列が帰る、ので自身でパース、キャストしないといけない。
for(let i = 0; i < cookies.length; i++){
    const cookie = cookies[i];
    cookie = cookie.replace(/^[]+/, '')
    if(cookie[0] === "max_results" && cookie.indexOf("max_results") === 0){
        max_results = parseInt(cookie[1]);
    }
}
```
このように、「最大値を取得する」という目的をもったブロックに、cookieのパースやキャストなどの細かい処理を含めると、可読性を損なう。そこで、`get_cookie()`という関数に切り出してみると、
```js
//例：cookieの扱い
const get_cookie = (name) => {
    const cookies = document.cookie.split(";") 
    for(let i = 0; i < cookies.length; i++){
        const cookie = cookies[i];
        cookie = cookie.replace(/^[]+/, '')
        if(cookie[0] === name && cookie.indexOf(name) === 0){
            return cookie[1];
        }
    }
    return null;
}
//利用
let max_results;
max_results = get_cookie("max_results");
```
ブロックの処理が明確になり、可読性が向上する。


# 10.8 やりすぎ

小さな関数に切り分けるのをやりすぎると、逆にコードの可読性が悪くなる。
例えば、以下のように関数に切り分けすぎて、各関数の実行結果を予想するのに、別の位置にある関数定義を複数回読まなければいけないようなケース

```python

user_info = {
    "name": "John",
    "age": 30,
    "address": "Tokyo",
    "phone": "090-1234-5678"
}
url = "http://example.com" + url_safe_encode(user_info)

def url_safe_encode(user_info):
    return url_safe_encode_str(user_info)

def url_safe_encode_str(user_info):
    return url_safe_encode_dict(user_info)

def url_safe_encode_dict(user_info):
    return url_safe_encode_list(user_info)

....
```
これでは、関数の定義パスを無駄に追いかける事になるので、可読性が悪くなる。

> 関数を作るという行為は、ごくわずかに（しかし確実に）コードを読むときのコストを挙げることになる